<center><h1>Stack Overflow Search Engine</h1></center>

<h1>1.Problem Statement:</h1>

<p>This problem involves building a search engine for StackOverflow questions and study different techniques to get the relevant and semantically similar results based on the user query.
</p>

<h1>2.Business objectives and constraints:</h1>

<ul>
<li>low-latency requirement.providing results in few second</li>
<li>Providing relevant resutls to the user</li>

#<h1>3.Data Collection</h1>

<p>Data obtained from Stack Overflow database with the help of Google BigQuery</p>

<ul>
  <li>
    Stcak Overflow dataset is publicly available in the google cloud
  </li>
    <li>
    This dataset have many table but we are focusing mainly on posts_questions and posts_answers
  </li>
      <li>
    In posts_questions will have information about questions like id of the question,topics of the question,id of the creator, date on which question was created etc.
  </li>
        <li>
    In posts_answers will have information about answers corresponding to a question.In this table one question might have more than one entry because of some questions will have multiple answers 
  </li>
  <li>
  By joining these two tables will get infromation what we required.but due to resource constraints, I restricted the data to only "Python" and "Java" related questions
  </li>
    <li>
    To get any data from google cloud datasets will have to use google's biq query,and to do that we should have google cloud account.
  </li>
<li>
Using google's biq query will get the data and store it in drive.
  </li>
</ul>

In [0]:
#importing packages
import pandas as pd
import numpy as np

#big query purpose
from google.colab import auth
from google.cloud import bigquery
from google.colab import drive

<h2>3.1 Authenticating google account to access Google public datasets</h2>

<p>To use Google's Big query we should authenticate our gmail id which has google cloud account.</p>

In [0]:
auth.authenticate_user() #authenticating user

<h2>3.2 Mounting the Drive (For saving data)</h2>

<p>Google Colab is a free cloud service for Python programming language.and we can access and store files from drive through this platform. </p>

In [0]:
drive.mount('/content/drive/', force_remount=True) #mouting the drive

Mounted at /content/drive/


<h2>3.3 Querying data using Google BigQuery</h2>

In [0]:
#declaring variable which are used in entire notebook
currentDirectory = "/content/drive/My Drive/pcase_study_1/"
dataDirectory = currentDirectory + "data/"

<p>To access a dataset from Google BigQuery, we need an cloud accont and we have to create a project in that account.</p>


In [0]:
project_id = 'cryptic-smile-255308' #project id (we can obtain from the google cloud platform)
client = bigquery.Client(project=project_id) #creating an BigQuery Client

job_config = bigquery.QueryJobConfig(use_legacy_sql=False,allowLargeResults=True) #Creating job configurations
                                                                #StackOverflow is an big dataset,so to get huge data we have to set allowLargeResults to True

# Perform a query.
QUERY = (
    '''
SELECT distinct q.id, q.title, q.body, q.tags, a.body as answers, a.score 
FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` where Score>0 ORDER BY View_Count desc) AS q 
INNER JOIN (SELECT * FROM `bigquery-public-data.stackoverflow.posts_answers`  where Score>0 ORDER BY Score desc) AS a ON q.id = a.parent_id 
where q.tags like '%python%' or q.tags like '%java|%'
''')
query_job = client.query(QUERY,job_config=job_config)  # API request
rows = query_job.result()  # Waits for query to finish


<p>we are getting data from two tables of stack over flow dataset
<ul>
<li>posts_questions: this table has information about questions in the stack overflow</li>
<li>posts_answers: this table has anwers to the question</li>
</p>

<p>so by joining these two tables will get infromation what we need.but due to resource constraints, I restricted the data to only "Python" and "Java" related questions</p>

<h2>3.4 Saving Query results into drive</h2>

<p>Google's Big query give result as iterator not as a dataframe,so will create dataframe with the Big query results.</p>

In [0]:
rows_list = []
for _row in rows:#reading rows of the query one after another
    rows_list.append([_row['id'], _row['title'], _row['body'], _row['tags'], _row['answers'], _row['score']])
    
#creating dataframe from the list
df = pd.DataFrame(rows_list,columns=['id','title','body','tags','answers','score'])
#saving dataFrame to csv
df.to_csv(dataDirectory + 'data.csv',encoding='utf-8',index=False)

In [0]:
df.head(3)

,id,title,body,tags,answers,score
0,6842245,converting date time to 24 hour format,<p>The time I get from the server is like <co...,java|datetime,<p>Try this:</p>\n\n<pre><code>String dateStr ...,112
1,1205375,Filter by property,<p>Is it possible to filter a Django queryset ...,python|django|orm,<p>Nope. Django filters operate at the databa...,66
2,1927544,"""For"" loop first iteration",<p>I would like to inquire if there is an eleg...,python|algorithm|iteration,<p>Something like this should work.</p>\n\n<pr...,61


In [0]:
print("Columns in the DataSet:",df.columns)
print("Number of records in the data: ",df.shape[0])

Columns in the DataSet: Index(['id', 'title', 'body', 'tags', 'answers', 'score'], dtype='object')
Number of records in the data:  1793879


<p>'data.csv' is almost 3.5gb file in which we have total of 1793879 records and we know that stack overflow will have more than one answer to a question.we will  handle this in data pre-process section</p>
<p> </p>

<p> Columns:

<ul>
<li>Id      : Unique Id of the question</li>
<li>title   : title of the question</li>
<li>body    : body of the question </li>
<li>tags    : topic of the question </li>
<li>answers : answer for the perticular question</li>

In [0]:
drive.flush_and_unmount()#to copy erything to drive